# Simple Item Based Collaborative Filtering
SonNG
Hanoi 2017/09/16

Item based Colaborative Filering là một thuật toán dựa trên sự tương đồng giữa các item với nhau để từ đó đưa ra sự gợi ý cho người dùng. 

### Bước 1: Đọc dữ liệu và tạo ma trận đầu vào

In [115]:
import numpy as np
import math

# Read the data to numpy array
with open('data/user_data.csv', 'r') as input_data:
    raw_data = input_data.read()
    data = [row.split(',')[1:] for row in raw_data.split('\n')][1:]
    data = np.array(data, dtype=np.float)

print(raw_data)
print()
print(data)

user,m1,m2,m3
u1,2,-1,3
u2,5,2,-1
u3,3,3,1
u4,-1,2,2

[[ 2. -1.  3.]
 [ 5.  2. -1.]
 [ 3.  3.  1.]
 [-1.  2.  2.]]


### Bước 2: Tính toán độ tương đồng giữa các bộ phim

Độ tương đồng ở đây được tính dựa trên khoảng cách cosin: 
* Giả sử ta tính độ tương đồng của 2 bộ phim m1 và m2, ta thấy rằng hai bộ phim này cùng được đánh giá bởi người dùng u2 và u3 (như trong dữ liệu trên). 
* Chúng ta sẽ tạo ra 2 item-vector v1 cho m1 và v2 cho m2 trong user-space (u2, u3).
    > v1 = 5 u2 + 3 u3 <br> v2 = 2 u2 + 3 u3
* Sau đó độ tương đồng cosin được tính bằng công thức:
    > cos(v1, v2) = (5*3 + 2*3) / sqrt[(25 + 9) * (4 + 9)] = 0.9037
* Tương tự như vậy, lặp lại với từng cặp phim khác nhau, ta có thể tính được độ tương đồng giữa chúng.**Kết quả ta sẽ thu được ma trận độ tương đồng giữa các bộ phim**

In [116]:
n_user, n_movie = data.shape
similarity_matrix =  np.diag(np.ones(n_movie))

upper = lambda a: a[0] * a[1]
lower = lambda b: b[0]*b[0] + b[1]*b[1]


def cal_cosin_distance(similarity_vector):
    upper_sum = np.sum(np.apply_along_axis(upper, 1, similarity_vector))
    lower_sum = math.sqrt(np.prod(np.apply_along_axis(lower, 0, similarity_vector)))
    distance = upper_sum / lower_sum
    return distance

# Loop through each movie pair to calculate the similarity
for m in range(n_movie):
    for n in range(m + 1, n_movie, 1):
        similarity_vector = data[np.where((data[:, m] != -1) * (data[:, n] != -1))][:, (m,n)]
        distance = cal_cosin_distance(similarity_vector)
        similarity_matrix[m, n] = distance
        similarity_matrix[n, m] = distance
        print("Movie 1: {} and Movie 2: {} have the similarity of: {} \n".format(m, n, distance))
print("Similarity matrix between movies:")
print(similarity_matrix)

Movie 1: 0 and Movie 2: 1 have the similarity of: 0.9037378388935388 

Movie 1: 0 and Movie 2: 2 have the similarity of: 0.7893522173763263 

Movie 1: 1 and Movie 2: 2 have the similarity of: 0.8682431421244593 

Similarity matrix between movies:
[[ 1.          0.90373784  0.78935222]
 [ 0.90373784  1.          0.86824314]
 [ 0.78935222  0.86824314  1.        ]]


### Bước 3: Tính toán rating của người dùng

* Từ ma trận tương đồng vừa tìm được, ta sẽ tính toán rating của người dùng dựa trên độ tương đồng của bộ phim mà người đó đã đánh giá với những bộ phim người đó chưa đánh giá.
* Cụ thể ví dụ như để tính toán rating của người dùng u1 cho bộ phim m2 ta có thể tính bằng công thưc sau: 
    > rating = ``(2 * 0.90373784 + 3 * 0.86824314) / (0.90373784 + 0.86824314) = 2.48998451``

In [113]:
def cal_rating(user_rating_vector, target_movie):
    upper = 0.
    lower = 0.
    for m in range(n_movie):
        if user_rating_vector[m] == -1 or m == target_movie: continue
        upper += user_rating_vector[m] * similarity_matrix[target_movie, m]
        lower += recommend_matrix[target_movie, m]
    rating = upper/lower
    return rating
    
unrated_movie = np.zeros((n_user, n_movie), dtype=np.float32)

# For each user and for each movie that user hadn't rated yet, predict rating for that user - movie pair
for u in range(n_user):
    for m in range(n_movie):
        if data[u, m] == -1:
            user_ratings = data[u, :]
            unrated_movie[u, m] = cal_rating(user_ratings, m)
print("Original data:")
print(data)
print()
print("Predict rating only data:")
print(unrated_movie)

Original data:
[[ 2. -1.  3.]
 [ 5.  2. -1.]
 [ 3.  3.  1.]
 [-1.  2.  2.]]

Predict rating only data:
[[ 0.          2.48998451  0.        ]
 [ 0.          0.          3.42860961]
 [ 0.          0.          0.        ]
 [ 2.          0.          0.        ]]


### Bước 4: Tìm ra bộ phim được dự đoán có rating cao nhất

* Ở bước cuối, ta sắp xếp kết quả trong ma trận dự đoán vừa tìm được và chọn ra kết quả có đánh giá cao nhất để gợi ý cho người dùng

In [114]:
# Find the max rating
recommend_movie = unrated_movie.argmax(axis=1)
for user, movie in enumerate(recommend_movie): 
    print("Recommend movie {} for user {}".format(movie, user))

Recommend movie 1 for user 0
Recommend movie 2 for user 1
Recommend movie 0 for user 2
Recommend movie 0 for user 3


[Linux Uncle Blog]: [https://ashokharnal.wordpress.com/2014/12/18/worked-out-example-item-based-collaborative-filtering-for-recommenmder-engine/]
#### Reference: 
* [Linux Uncle Blog][] Linux Uncle Blog